In [19]:
import time
import random

import matplotlib.pyplot as plt
import osmnx as ox
import networkx as nx
import folium
import geopandas as gpd
import shapely.geometry

import transit as tr

Задаем константы


In [2]:
GTFSpath = r"D:\Python_progs\Feeds\Klapeida" # Путь к папке с GTFS файлами
departure_time_input = "18:00:00" # Время отправления в формате HH:MM:SS
day = 'monday' # День недели
time_zero = time.time()

plot_graph = False # Не работает
save_city_graph = False # Сохранять граф города в файл
plot_path_in_browser = False # Отобразить маршрут при помощи Folium в браузере
plot_path_folium = True # Отобразить маршрут при помощи Folium в Ipython Notebook

Загружаем комбинированный граф GTFS и OSM

In [3]:
G_combined, stops = tr.feed_to_graph(GTFSpath, departure_time_input, 
                                      day, duration_seconds=3600*3, 
                                      save_to_csv = True)

GTFS data loaded
12181 of 72974 trips retained
Выпуклая оболочка построена, начинается загрузка графа улиц
Граф улиц загружен
Начинается соединение графов
Число узлов: 32002
Число ребер: 83167
Соединение завершено


Выбираем рандомные узлы графа для расчета времени в пути. 
По координатам потом удобно сверять время с Гугл картами/Яндексом  

p.s. Главное отключить метро :) 
p.p.s В гугл картах лучше сравнивать не время в пути, а время прибытия, т.к. они не показывают время ожидания



In [4]:
source = random.choice(list(G_combined.nodes))
target = random.choice(list(G_combined.nodes))

print("Source: ", source, "coordinates", G_combined.nodes[source]['y'], G_combined.nodes[source]['x'])
print("Target: ", target, "coordinates", G_combined.nodes[target]['y'], G_combined.nodes[target]['x'])

Source:  4306315552 coordinates 55.7509051 21.3289425
Target:  1838685887 coordinates 55.8820097 21.2221105


In [5]:
def main():
    sample_time = time.time()
    
    path, arrival_time, travel_time, used_routes = tr.time_dependent_dijkstra(
                            graph = G_combined, 
                            source = source, 
                            target = target, 
                            start_time = tr.parse_time_to_seconds(departure_time_input),
                            track_used_routes=True,
                        )

    #print(f"Path: {path}")
    print('Used routes:', used_routes)
    print(f"Arrival time at destination: {tr.parse_seconds_to_time(arrival_time)} in {tr.parse_seconds_to_time(travel_time)}")

    print(f"Time elapsed: {time.time() - sample_time} seconds")

    return path, arrival_time, travel_time

path, arrival_time, travel_time = main()

Used routes: {None}
Arrival time at destination: 21:58:21 in 03:58:21
Time elapsed: 0.1914987564086914 seconds


In [6]:
G_path = nx.subgraph(G_combined, path)

graph = nx.MultiDiGraph(G_path)

G_nodes, G_edges = ox.graph_to_gdfs(graph)

frame_center_lat = stops['stop_lat'].mean()
frame_center_lon = stops['stop_lon'].mean()

if plot_path_in_browser:
    tr.plot_path_browser(graph, stops)

m2 = folium.Map(location=[frame_center_lat, frame_center_lon], 
                zoom_start=12,
                width='100%', height='100%')

G_edges.explore(m = m2)

G_nodes.explore(m = m2,
                column = 'type',
                cmap = 'rainbow')

In [21]:
sa_source = random.choice(list(G_combined.nodes))

In [22]:
polygon, points = tr.create_service_area(G_combined, sa_source, start_time=64800, cutoff = 1200, buffer_radius=100)

In [23]:
point = shapely.geometry.Point(G_combined.nodes[sa_source]['x'], G_combined.nodes[sa_source]['y'])
point_gdf = gpd.GeoDataFrame(geometry=[point])

frame_center_lat = stops['stop_lat'].mean()
frame_center_lon = stops['stop_lon'].mean()

m = folium.Map(location=[frame_center_lat, frame_center_lon], 
                zoom_start=12,
                width='100%', height='100%')

polygon.explore(m = m)
#polygon2.explore(m = m, color = 'green')
#polygon3.explore(m = m, color = 'red')

point_gdf.explore(m = m, color = 'red')

In [24]:
path2, arrival_time2, travel_time2, used_routes2 = tr.time_dependent_dijkstra(
                            graph = G_combined, 
                            source = 5741, 
                            target = 7031, 
                            start_time = tr.parse_time_to_seconds(departure_time_input),
                            track_used_routes=True,
                        )

#print(f"Path: {path}")
print('Used routes:', used_routes2)
print(f"Arrival time at destination: {arrival_time2} in {travel_time2}")
print(f"Arrival time at destination: {tr.parse_seconds_to_time(arrival_time2)} in {tr.parse_seconds_to_time(travel_time2)}")

Used routes: {None, 'klaipeda_bus_21A', 'klaipeda_bus_M8', 'klaipeda_bus_18', 'klaipeda_bus_5', 'klaipeda_bus_8'}
Arrival time at destination: 67887.34075281981 in 3087.340752819815
Arrival time at destination: 18:51:27 in 00:51:27


In [25]:
'''def main():
    G_combined, city_name, departure_time_seconds, stops = create_GTFS_graph(GTFSpath, departure_time_input, 
                                                                            day, 
                                                                            duration_seconds=3600*3, 
                                                                            save_to_csv = True)

cProfile.run("main()")'''

'def main():\n    G_combined, city_name, departure_time_seconds, stops = create_GTFS_graph(GTFSpath, departure_time_input, \n                                                                            day, \n                                                                            duration_seconds=3600*3, \n                                                                            save_to_csv = True)\n\ncProfile.run("main()")'

In [26]:
"""times_list = []
startt = parse_time_to_seconds(departure_time_input)
endt = startt + 3600
for i in range(startt, endt, 360):
    try:
        path, arrival_time, travel_time, used_routes = time_dependent_dijkstra_2(
                                        graph = G_combined, 
                                        source = source, 
                                        target = target, 
                                        start_time = i,
                                        track_used_routes=True)
                                    
        times_list.append(tuple([i, travel_time]))
    except:
        continue
    
print(times_list)

plt.figure(figsize=(14,7))
plt.plot(*zip(*times_list))

plt.xlabel('Departure Time')
plt.ylabel('Travel Time')
plt.show()"""

"times_list = []\nstartt = parse_time_to_seconds(departure_time_input)\nendt = startt + 3600\nfor i in range(startt, endt, 360):\n    try:\n        path, arrival_time, travel_time, used_routes = time_dependent_dijkstra_2(\n                                        graph = G_combined, \n                                        source = source, \n                                        target = target, \n                                        start_time = i,\n                                        track_used_routes=True)\n                                    \n        times_list.append(tuple([i, travel_time]))\n    except:\n        continue\n    \nprint(times_list)\n\nplt.figure(figsize=(14,7))\nplt.plot(*zip(*times_list))\n\nplt.xlabel('Departure Time')\nplt.ylabel('Travel Time')\nplt.show()"